# Investigation of the reliability of borrowers

The customer is the credit department of the bank. It is necessary to understand whether the marital status and the number of children of the client affect the fact of repayment of the loan on time. Input data from the bank — statistics on the solvency of customers.

The results of the study will be taken into account when building a model of ** credit scoring** — a special system that evaluates the ability of a potential borrower to repay a loan to the bank.

## Step 1. Open the data file and study the general information<a name="step_1"></a>

In [1]:
import pandas as pd

In [2]:
#Вывод
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Conclusion**

There are negative age values.The 'Level of education of the client' is incorrectly specified, there are: "secondary" and "HIGHER" .

##Step 2. Data Preprocessing<a name="step_2"></a>

### Processing of omissions

In [3]:
df['family_status'].isna().mean()

0.0

In [4]:
df['purpose'].isna().mean()

0.0

In [5]:
df['children'].isna().mean()

0.0

In [6]:
df.loc[df.children == -1, 'children'] = 1

In [7]:
df.loc[df.children == 20, 'children'] = 2

In [8]:
df['debt'].isna().mean()

0.0

In [9]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [10]:
df['total_income'].isna().mean()

0.10099883855981417

In [11]:
df['total_income_check'] = df['total_income'].isnull()

In [12]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [13]:
df['total_income'].isna().mean()

0.10099883855981417

In [14]:
df['total_income_check'] = df['total_income'].isnull()
df.loc[df['total_income_check'] == True ,:]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_check
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,True
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,True
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,True
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,True
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем,True
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба,True
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости,True
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости,True


In [15]:
total_income_mean = df['total_income'].mean()

In [16]:
df['total_income'].isna().mean()

0.10099883855981417

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children              21525 non-null int64
days_employed         19351 non-null float64
dob_years             21525 non-null int64
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null int64
total_income          19351 non-null float64
purpose               21525 non-null object
total_income_check    21525 non-null bool
dtypes: bool(1), float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [20]:
df['total_income']

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64

We use the 'as type' function, because our columns have a pure 'float64' format, i.e. translation to 'int' will be safe.

### Processing duplicates

In [22]:
df.duplicated().sum()

54

There are 54 duplicate lines. Remove duplicate values
Reason:entering application data multiple times

In [23]:
df['education'] = df['education'].str.lower()

In [24]:
df=df.drop_duplicates().reset_index(drop=True)

In [25]:
df.duplicated().sum()

0

In [26]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_check
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,False
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,False
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,False
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,False
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,False
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,False
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,False
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,False
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,False
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,False


### Lemmatization

In [27]:
#Изучим столбец "цель получения кредита"
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

There are many objects that have a similar meaning, but written differently.

Let's use the pymystem3 module and lemmatize the list obtained above.

In [28]:
from pymystem3 import Mystem
m = Mystem()

In [29]:
purpose_dict = df['purpose'].unique()

In [30]:
lemmas_dict_temp = []

In [31]:
for purpose in purpose_dict:
    lemmas = m.lemmatize(purpose)
    for l in lemmas:
        lemmas_dict_temp.append(l)

In [32]:
#Цели кредита
print(lemmas_dict_temp)

['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n', 'на', ' ', 'проведение', ' ', 'свадьба', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n', 'покупка', ' ', 'недвижимость', '\n', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '\n', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '\n', 'недвижимость', '\n', 'строительство', ' ', 'недвижимость', '\n', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '\n', 'жилье', '\n', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n', 'автомобиль', '\n', 'заниматься', ' ', 'образование', '\n', 'сделка', ' '

In [33]:
def class_purpose(data):
    data = m.lemmatize(data)
    if 'недвижимость' in data:
        return 'недвижимость'
    if 'жилье' in data:
        return 'недвижимость'
    if 'свадьба' in data:
        return 'свадьба'
    if 'автомобиль' in data:
        return 'автомобиль'
    if 'образование,'  in data:
        return 'образование'
    if  'образование' in data:
        return 'образование'
   
    return 'другое'

In [34]:
df['purpose_type'] = df['purpose'].apply(class_purpose)

In [35]:
df['purpose_type'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_type, dtype: int64

**Conclusion**

The most frequent categories of taking out a loan: real estate, car, education, wedding.

### Categorization of data

Let 's formulate the principle of classification:

Let's categorize the data by the presence of children.

In [36]:

df.groupby('children')['debt'].agg(['count','mean'])

,count,mean
children,,
0,14091,0.075438
1,4855,0.091658
2,2128,0.094925
3,330,0.081818
4,41,0.097561
5,9,0.000000


We will perform the categorization of data on the earnings of customers.

In [37]:
def income_group(income):
     
        if income <= 50000:
                return 'низкий'
        if income <= 100000 and income >= 50001 :
                return 'средний'
        if income <= 160000 and income >= 100001 :
                return 'выше среднего'
        return 'высокий'

In [38]:
df['total_income_id'] = df['total_income'].apply(income_group)

In [39]:
df.groupby('total_income_id')['debt'].agg(['count','mean'])

,count,mean
total_income_id,,
высокий,10321,0.078287
выше среднего,6670,0.086807
низкий,372,0.061828
средний,4091,0.080909


Let's categorize the data by family status.

In [40]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [41]:
df.groupby('family_status')['debt'].agg(['count','mean'])

,count,mean
family_status,,
Не женат / не замужем,2810,0.097509
в разводе,1195,0.071130
вдовец / вдова,959,0.065693
гражданский брак,4151,0.093471
женат / замужем,12339,0.075452


We will categorize the data for the reason of taking a loan.

In [42]:
df.groupby('purpose_type')['debt'].agg(['count','mean'])

,count,mean
purpose_type,,
автомобиль,4306,0.093590
недвижимость,10811,0.072334
образование,4013,0.092200
свадьба,2324,0.080034


**Conclusion**

The dataframe was categorized according to the following data:
by the number of children ('children');
by salary level ('salary');
by marital status ('family_status');
due to taking a loan('purpose_type');


## Step 3. Answer the questions

- Is there a relationship between having children and repayment of the loan on time?

Most of the delays are from a family with two children.

- Is there a relationship between marital status and repayment of the loan on time?

Widowers/widows turned out to be the most responsible clients of the bank, unmarried/unmarried clients have large debts

- Is there a relationship between the level of income and repayment of the loan on time?

There are the least debtors among low-income clients, the most debtors among above-average income clients

- How do the different purposes of the loan affect its repayment on time?

There are many debtors who take out a loan for car operations

## Step 4. General output

The repayment of the loan depends on many factors.All dependencies by customer categories do not differ by much.